In [26]:
from __future__ import print_function
import torch.utils.data as data
import os
import os.path
import torch
import numpy as np
import sys
from tqdm import tqdm 
import json
import glob
from plyfile import PlyData, PlyElement

In [19]:
npoints = 2500
root = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data'
cat = {}
data_augmentation = True
classification = False
split = 'train'
seg_classes = {}
class_choice = None

In [20]:
catfile = os.path.join(root, 'synsetoffset2category.txt')

with open(catfile, 'r') as f:
    for line in f:
        ls = line.strip().split()
        cat[ls[0]] = ls[1]

In [21]:
if not class_choice is None:
    cat = {k: v for k, v in cat.items() if k in class_choice}

In [22]:
print(cat)

{'Head': '02691156'}


In [56]:
id2cat = {v: k for k, v in cat.items()}
print(id2cat)

{'02691156': 'Head'}


In [57]:
for item in cat:
    meta[item] = []

In [58]:
print(meta)

{'Head': []}


In [59]:
for category, values in cat.items():
    print(values)

02691156


In [60]:
for key, value in cat.items():
    newDir = os.path.join(root, value)
    for file in glob.glob(newDir + '/*.txt'):
        for category in cat.values():
            meta[id2cat[category]].append(file)

In [61]:
list(meta.values())[0], list(meta.keys())[0]

(['/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/5.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/2.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/1.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/6.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/3.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/7.txt',
  '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/4.txt'],
 'Head')

In [62]:
datapath = []
for item in cat:
    for fn in meta[item]:
        datapath.append((item, fn))

In [63]:
print(datapath)

[('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/5.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/2.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/1.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/6.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/3.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/7.txt'), ('Head', '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/head_data/02691156/4.txt')]


In [64]:
print(len(datapath))

7


In [65]:
classes = dict(zip(sorted(cat), range(len(cat))))
print(classes)

{'Head': 0}


In [66]:
with open(os.path.join(os.getcwd(), '../misc/num_seg_classes_head.txt'), 'r') as f:
    for line in f:
        ls = line.strip().split()
        seg_classes[ls[0]] = int(ls[1])
num_seg_classes = seg_classes[list(cat.keys())[0]]
print(seg_classes, num_seg_classes)

{'Head': 2} 2


In [ ]:
##################################################################################################3

In [68]:
index = 0

In [72]:
fn = datapath[index]
cls = classes[datapath[index][0]]
data = np.loadtxt(fn[1])
point_set = data[:, 0:3].astype(np.float32)
seg = data[:, -1].astype(np.int64)

In [73]:
print(point_set, cls, seg)

[[-9.81000e-03  0.00000e+00  1.44000e-04]
 [-1.62160e-02  0.00000e+00  1.44000e-04]
 [-2.90280e-02 -4.29000e-04  2.72000e-04]
 ...
 [-1.03814e-01  2.17127e-01  1.77469e-01]
 [-9.80260e-02  2.17264e-01  1.78167e-01]
 [-9.45360e-02  2.17177e-01  1.75970e-01]] 0 [0 0 0 ... 1 1 1]


In [74]:
print(point_set.shape, seg.shape)

(9604, 3) (9604,)


In [75]:
choice = np.random.choice(len(seg), npoints, replace=True)
# resample
point_set = point_set[choice, :]

point_set = point_set - np.expand_dims(np.mean(point_set, axis = 0), 0) # center
dist = np.max(np.sqrt(np.sum(point_set ** 2, axis = 1)),0)
point_set = point_set / dist # scale

In [76]:
print(point_set)

[[ 0.33044326 -0.07626065 -0.10912184]
 [ 0.04921341  0.06585306 -0.44915983]
 [ 0.38648164  0.309005    0.00827159]
 ...
 [ 0.30727243 -0.05384934  0.23693164]
 [-0.4008455   0.30272114 -0.04938204]
 [ 0.12243243  0.33296573  0.3938259 ]]


In [77]:
seg = seg[choice]
point_set = torch.from_numpy(point_set)
seg = torch.from_numpy(seg)
cls = torch.from_numpy(np.array([cls]).astype(np.int64))

In [78]:
print(point_set, cls, seg)

tensor([[ 0.3304, -0.0763, -0.1091],
        [ 0.0492,  0.0659, -0.4492],
        [ 0.3865,  0.3090,  0.0083],
        ...,
        [ 0.3073, -0.0538,  0.2369],
        [-0.4008,  0.3027, -0.0494],
        [ 0.1224,  0.3330,  0.3938]]) tensor([0]) tensor([0, 0, 0,  ..., 0, 1, 0])
